In [1]:
from diffeq import *

## Basic vector operations

In [2]:
from diffeq.utils.vectors import *
A = vector(x = 10, y = 10)
B = vector(x = 3, y = -1, z = 1)

print('A:', A, sep = '\n'*2)
print('B:', B, sep = '\n'*2)
print('A + B:', A + B, sep = '\n'*2)
print('A - B:', A - B, sep = '\n'*2)
print('A @ B:', A @ B, sep = '\n'*2)
print('A * B:', A * B, sep = '\n'*2)
print('A / B:', A / B, sep = '\n'*2)
print('A ** B:', A ** B, sep = '\n'*2)
print('A**2:', A**2, sep = '\n'*2)
print('2**A:', A**2, sep = '\n'*2)
print('A@A:', A@A, sep = '\n'*2)

A:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │10     │
└─────────────┘
B:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │3      │
│y    │-1     │
│z    │1      │
└─────────────┘
A + B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│z    │1.0     │
│y    │9       │
│x    │13      │
└──────────────┘
A - B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│z    │-1.0     │
│y    │11       │
│x    │7        │
└───────────────┘
A @ B:

20
A * B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│z    │0.0     │
│y    │-10     │
│x    │30      │
└──────────────┘
A / B:

┌─────────────────────────────┐
│axis │value                  │
├─────┼───────────────────────┤
│z    │0.0                    │
│y    │-10.0                  │
│x    │3.3333333333333335     │
└─────────────────────────────┘
A ** B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│z    │0.0      │
│y    │0.1      │
│x    │1000     │
└───────────────┘
A**2:

┌──────────

## Vector functions

In [3]:
@vector_function
def foo(x, y):
    return vector(
        x=10*y*x + y,
        y=x,
    )

v = vector(x = 10, y = 11)
print('value of input vector: ', v, sep = '\n')
print('function form: ', sep = '\n')
print(foo)
print('function output: ', sep = '\n')
print(foo(v))
print("function's yacobian: ", sep = '\n')
print(foo.yacobian)
print("function's yacobian value: ", sep = '\n')
print(foo.yacobian(v))

value of input vector: 
┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │11     │
└─────────────┘
function form: 
┌────────────────────────┐
│axis │function          │
├─────┼──────────────────┤
│x    │(y*x*10+y)        │
│y    │x                 │
└────────────────────────┘
function output: 
┌───────────────┐
│axis │value    │
├─────┼─────────┤
│x    │1111     │
│y    │10       │
└───────────────┘
function's yacobian: 
┌────────────────────────────┐
│axis     │function          │
├─────────┼──────────────────┤
│dx_dx    │(y*10)            │
│dx_dy    │(x*10+1.0)        │
│dy_dx    │1.0               │
│dy_dy    │0.0               │
└────────────────────────────┘
function's yacobian value: 
┌────────────────────┐
│axis     │value     │
├─────────┼──────────┤
│dx_dx    │110       │
│dx_dy    │101.0     │
│dy_dx    │1.0       │
│dy_dy    │0.0       │
└────────────────────┘


composition:

In [4]:
@vector_function
def foo1(x, y):
    return vector(
        x=10*y*x + y,
        y=-x + (-1),
    )

@vector_function
def foo2(x, y):
    return vector(
        x = 3*y - x,
        y = x - 1j
    )

In [5]:
@vector_function
def composed(x, y):
    return foo2(foo1(vector(x = x, y = y)))

A = vector(x = 1, y = 2)
print(composed)
print(composed(A))
print(foo2(foo1(A)))

print(composed.yacobian)

┌─────────────────────────────────────────┐
│axis │function                           │
├─────┼───────────────────────────────────┤
│x    │(((-x)+-1)*3+(-(y*x*10+y)))        │
│y    │(y*x*10+y+-1j)                     │
└─────────────────────────────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌────────────────────────────────┐
│axis     │function              │
├─────────┼──────────────────────┤
│dx_dx    │((-y*10)+-3.0)        │
│dx_dy    │0.0                   │
│dy_dx    │(y*10)                │
│dy_dy    │(x*10+1.0)            │
└────────────────────────────────┘


In [6]:
print(composed.div)

┌───────────────────────────────────┐
│axis   │function                   │
├───────┼───────────────────────────┤
│div    │((-y*10)+x*10+-2.0)        │
└───────────────────────────────────┘


In [7]:
print(composed.div.yacobian)

┌───────────────────────┐
│axis       │function   │
├───────────┼───────────┤
│ddiv_dx    │10         │
│ddiv_dy    │-10        │
└───────────────────────┘


## System of differential equations

In [8]:
solver = rk4_integrator(0.1)
sys = system(
    vector_function(lambda x, y:vector(x = -y, y = x)), solver, initials = vector(x = 1, y = 0)
)
results = sys.run(10)

In [9]:
print(results)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│axis    │value                                                                                                                                                                                              │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│x       │[1, 0.540302967116884, -0.41614526873411317, -0.9899919390660014, -0.6536459531899061, 0.28365810583410267, 0.9601684949770736, 0.7539057070501398, -0.14549338089595665, -0.9111266132598114]     │
│y       │[0, 0.8414704778002743, 0.9092979917935007, 0.1411224448434349, -0.7568001143085917, -0.9589251198182553, -0.27942016563257394, 0.6569818976735576, 0.989358664182

## Visualization